In [1]:
import json
import pandas as pd
import os
import csv
import numpy
import requests
from pprint import pprint

api_key = "a2rOTe8PfBiJBwTOSlRbteARqAEa0s6DsRyoSOOF"
url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json?'

search = 'school.degrees_awarded.predominant=3&_fields=id,school.name,latest.cost.tuition.in_state,latest.cost.tuition.out_of_state,school.region_id,latest.earnings.10_yrs_after_entry.median,latest.earnings.6_yrs_after_entry.median,latest.repayment.5_yr_repayment.completers_rate,latest.repayment.7_yr_repayment.completers_rate,latest.repayment.3_yr_repayment.completers_rate'

search_url = url + search + '&api_key=' + api_key

response = requests.get(search_url)
response_json = response.json()
# pprint (response.json())


In [47]:
# school_ids_list = []
# schools_list = []
# regions_list = []

# for page in range (0,105):
#     search_url = f'{url}{search}&page={page}&api_key={api_key}'
#     response = requests.get(search_url)
#     response_json = response.json()
       
#     for num in range(0,20):
#         school_id = response_json['results'][num]['id']
#         school_ids_list.append(school_id)
#         school_name = response_json['results'][num]['school.name']
#         schools_list.append(school_name)  
#         region_id = response_json['results'][num]['school.region_id']
#         regions_list.append(region_id)
                
# # print (len(school_ids_list))
# # print (len(schools_list))
# # print (len(regions_list))

# schoolinfo_df = pd.DataFrame({
#     "School ID": school_ids_list,
#     "School": schools_list,
#     "Region ID": regions_list
# })

# schoolinfo_df.to_csv("schoolinfo.csv", index=False, header=False)
# schoolinfo_df                    
            


KeyError: 'results'

In [52]:
# school_ids_list = []
# tuition_in_state_list = []
# tuition_out_state_list = []

# for page in range (0,105):
#         search_url = f'{url}{search}&page={page}&api_key={api_key}'
#         response = requests.get(search_url)
#         response_json = response.json()
        
#         for num in range(0,20):
#             school_id = response_json['results'][num]['id']
#             school_ids_list.append(school_id)                
#             tuition_in = response_json['results'][num]['latest.cost.tuition.in_state']
#             tuition_in_state_list.append(tuition_in)  
#             tuition_out = response_json['results'][num]['latest.cost.tuition.out_of_state']
#             tuition_out_state_list.append(tuition_out)
                
# school_tuition_df = pd.DataFrame({
#     "School ID": school_ids_list,
#     "Tuition In of State": tuition_in_state_list,
#     "Tuition Out of State": tuition_out_state_list
# })
                
# school_tuition_df.to_csv("school_tuition.csv", index=False, header=False)
# school_tuition_df                

In [8]:
schoolcsvpath = os.path.join('schoolinfo.csv')
tuitioncsvpath = os.path.join('school_tuition.csv')

school_pd_df = pd.read_csv(schoolcsvpath, encoding="utf-8")
tuition_pd_df = pd.read_csv(tuitioncsvpath, encoding="utf-8")

In [9]:
school_pd_df.head()

,School Id,School Name,Region Id
0,445188,University of California-Merced,8
1,446792,Sanford-Brown College-Las Vegas,8
2,446808,Sanford-Brown College-Seattle,8
3,446048,Ave Maria University,5
4,446668,The Art Institute of Ohio-Cincinnati,3


In [10]:
tuition_pd_df.head()

,School Id,Tuition In State,Tuition Out of State
0,445188,13262.0,39944.0
1,446792,NaN,NaN
2,446808,NaN,NaN
3,446048,19128.0,19128.0
4,446668,NaN,NaN


In [11]:
school_tuition_merge = pd.merge(school_pd_df, tuition_pd_df, on='School Id', how="outer")
school_tuition_merge.head()

,School Id,School Name,Region Id,Tuition In State,Tuition Out of State
0,445188,University of California-Merced,8,13262.0,39944.0
1,446792,Sanford-Brown College-Las Vegas,8,NaN,NaN
2,446808,Sanford-Brown College-Seattle,8,NaN,NaN
3,446048,Ave Maria University,5,19128.0,19128.0
4,446668,The Art Institute of Ohio-Cincinnati,3,NaN,NaN
